<a href="https://colab.research.google.com/github/imannost/svoyak-net/blob/main/svoyak_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>